# Advanced Querying Mongo

Importing libraries and setting up connection

In [33]:
import pandas as pd
import pymongo
pd.options.display.max_columns = None
client = MongoClient()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [7]:
ironhack = client['ironhack']
companies = ironhack.companies

babelgum = list(companies.find({'name':'Babelgum'}, {'name':1, '_id':0}))
babelgum

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [36]:
more_5000 = pd.DataFrame(companies.find({'number_of_employees' : {'$gt':5000}}).sort('number_of_employees', 1).limit(20))
more_5000

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297e49,Nintendo,nintendo,http://www.crunchbase.com/company/nintendo,http://nintendo.com,,,NintendoAmerica,games_video,5080,1889.0,9.0,23.0,None,None,None,None,"games, console",,noalegal@noa.nintendo.com,1-800-255-3700,,Sat Dec 22 13:27:00 UTC 2007,Thu Sep 05 12:41:16 UTC 2013,"<p>Nintendo, a technology company widely known...","{'available_sizes': [[[150, 41], 'assets/image...","[{'name': 'Wii', 'permalink': 'wii'}]","[{'is_past': False, 'title': 'President', 'per...",[],[],$0,[],[],None,[],"[{'description': None, 'address1': None, 'addr...","[{'id': 17591, 'description': '3D For Everyone...",None,[],[],[],[]
1,52cdef7e4bab8bd67529ba2e,Hexaware Technologies,hexaware-technologies,http://www.crunchbase.com/company/hexaware-tec...,http://www.hexaware.com,http://blogs.hexaware.com,http://blogs.hexaware.com/feed,None,other,5200,1990.0,NaN,NaN,None,None,None,None,"it-services--bpo-services, peoplesoft-services...",None,corporatemarketing@hexaware.com,+91 44 4200 1600,IT & BPO service Provider,Fri May 08 17:46:58 UTC 2009,Mon May 18 20:12:25 UTC 2009,<p>Hexaware Technologies is a leading global p...,None,[],"[{'is_past': False, 'title': 'Founder & Execut...","[{'competitor': {'name': 'Capgemini', 'permali...",[],$0,[],[],None,"[{'price_amount': 34300000, 'price_currency_co...","[{'description': 'United States of America', '...",[],None,[],[],[{'external_url': 'http://www.forrester.com/Re...,[]
2,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004.0,2.0,1.0,None,None,None,,"facebook, college, students, profiles, network...",,,,Social network,Fri May 25 21:22:15 UTC 2007,Thu Nov 21 19:40:55 UTC 2013,<p>Facebook is the world&#8217;s largest socia...,"{'available_sizes': [[[150, 61], 'assets/image...","[{'name': 'Facebook Platform', 'permalink': 'f...","[{'is_past': False, 'title': 'Founder and CEO,...","[{'competitor': {'name': 'MySpace', 'permalink...","[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarters', 'address1': '...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...",[],"[{'available_sizes': [[[150, 68], 'assets/imag...",[{'external_url': 'http://latimesblogs.latimes...,[]
3,52cdef7e4bab8bd67529a453,OpenText,opentext,http://www.crunchbase.com/company/opentext,http://www.opentext.com,http://blogs.opentext.com,http://feeds.feedburner.com/OpenTextBlogs,OpenText,software,5300,1991.0,10.0,6.0,None,None,None,None,"crm, bpm, ecm, ediscovery",artesia,support@opentext.com,800-540-7292,,Wed Nov 26 07:51:26 UTC 2008,Mon Jan 06 03:18:08 UTC 2014,<p>Open Text Corporation (Open Text) is an ind...,"{'available_sizes': [[[150, 32], 'assets/image...",[],"[{'is_past': False, 'title': 'President & CEO'...",[],[],$0,[],[],None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarter', 'address1': '2...",[],"{'valuation_amount': None, 'valuation_currency...",[],[],[],[]
4,52cdef7d4bab8bd675298b89,LSI,lsi,http://www.crunchbase.com/company/lsi,http://www.lsi.com,,,LSICorporation,hardware,5400,1980.0,NaN,NaN,None,None,None,None,"1394, pci, serdes, mpeg, video, dsp, npu, cont...",LSI Logic,,408-433-8000,storage and networ

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [40]:
cond1 = {'founded_year': {'$gte':2000}}
cond2 = {'founded_year': {'$lte':2005}}

founded_2000_2005 = pd.DataFrame(companies.find({'$and': [cond1, cond2]}, {'name':1, 'founded_year':1, '_id':0}).limit(3))
founded_2000_2005

,name,founded_year
0,Zoho,2005
1,Digg,2004
2,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [43]:
cond1 = {'ipo.valuation_amount': {'$gt':100000000}}
cond2 = {'founded_year': {'$lt':2010}}
valuation_founded = list(companies.find({'$and': [cond1, cond2]}, {'name':1, 'ipo':1, '_id':0}).limit(3))
valuation_founded

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [46]:
cond1 = {'number_of_employees': {'$lt':1000}}
cond2 = {'founded_year': {'$lt':2005}}
employees_founded = pd.DataFrame(companies.find({'$and': [cond1, cond2]}).sort('number_of_employees', -1).limit(10))
employees_founded

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd675298933,Infinera Corporation,infinera,http://www.crunchbase.com/company/infinera,http://www.infinera.com,,,InfineraCorp,network_hosting,974,2000,NaN,NaN,None,None,None,None,None,Zepton Networks,,408-572-5200,Optical Networking Systems,Tue Apr 29 19:43:09 UTC 2008,Tue Jan 10 22:07:21 UTC 2012,<p>Infinera Corporation provides optical netwo...,"{'available_sizes': [[[150, 38], 'assets/image...",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 20267, 'description': 'Infinera PICs S...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 112], 'assets/ima...",[],[]
1,52cdef7e4bab8bd67529ac95,NorthPoint Communications Group,northpoint-communications-group,http://www.crunchbase.com/company/northpoint-c...,,,,,network_hosting,948,1997,NaN,NaN,None,None,None,None,None,,,415-403-4003,data transport services,Fri Jan 16 03:27:17 UTC 2009,Sun Jun 20 20:07:21 UTC 2010,"<p>NorthPoint Communications Group, Inc. provi...",None,[],"[{'is_past': True, 'title': 'SVP, Business Dev...",[],[],$0,[],[],None,[],"[{'description': 'HQ', 'address1': '303 Second...",[],None,[],[],[],[]
2,52cdef7f4bab8bd67529be17,888 Holdings,888-holdings,http://www.crunchbase.com/company/888-holdings,http://www.888holdingsplc.com,,,,games_video,931,1997,NaN,NaN,None,None,None,None,None,,info@888holdingsplc.com,+350 20049800,online gaming entertainment company,Sat Jun 06 18:58:57 UTC 2009,Tue Jun 15 03:02:26 UTC 2010,"<p>888 Holdings Public Limited Company, throug...","{'available_sizes': [[[150, 101], 'assets/imag...",[],"[{'is_past': False, 'title': 'User Expiriance ...",[],[],$0,[],[],None,"[{'price_amount': 18000000, 'price_currency_co...",[],[],None,[],"[{'available_sizes': [[[143, 150], 'assets/ima...",[],[]
3,52cdef7c4bab8bd6752986a2,Forrester Research,forrester-research,http://www.crunchbase.com/company/forrester-re...,http://forrester.com,http://blogs.forrester.com/colony/,http://blogs.forrester.com/colony/index.rdf,,consulting,903,1983,NaN,NaN,None,None,None,None,,,press@forrester.com,(617) 613-5730,,Sun Mar 30 16:42:50 UTC 2008,Fri Jul 19 10:09:08 UTC 2013,<p>Forrester Research is an independent techno...,"{'available_sizes': [[[121, 59], 'assets/image...",[],"[{'is_past': False, 'title': 'Researcher to th...",[],[],$0,[],[],None,"[{'price_amount': 4750000, 'price_currency_cod...","[{'description': None, 'address1': '400 Techno...","[{'id': 36801, 'description': 'Forrester: $2.1...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...",[{'external_url': 'http://blogs.forrester.com/...,[]
4,52cdef7e4bab8bd67529af6d,SonicWALL,sonicwall,http://www.crunchbase.com/company/sonicwall,http://www.sonicwall.com,,,sonicwall,security,900,1991,2.0,NaN,None,None,None,None,,None,sales@sonicwall.com,+1 408.745.9600,,Sat Jan 31 21:37:20 UTC 2009,Thu Sep 22 21:27:37 UTC 2011,<p>SonicWALL is a private company that provide...,"{'available_sizes': [[[149, 45], 'assets/image...",[],"[{'is_past': False, 'title': 'VP and CFO', 'pe...",[],[],$0,[],[],"{'price_amount': 717000000, 'price_currency_co...","[{'price_amount': 25000000, 'price_currency_co...","[{'description': 'HQ', 'address1': '2001 Logic...","[{'id': 15615, 'description': 'SonicWALL Revol...",None,"[{'embed_code': '<iframe width=""480"" height=""3...",[],[],[]
5,52cdef7e4bab8bd6752

### 6. All the companies that don't include the `partners` field.

In [62]:
"""
All the companies have the 'partners' field present, as can be verify running the following line:
    no_partners = pd.DataFrame(companies.find({'partners': {'$exists': False}}).limit(3))
So I've assumed that what it's asked is to show all the companies with the 'partners' field in blank
"""
no_partners = pd.DataFrame(companies.find({'partners': {'$exists': True, '$ne':''}}).limit(3))
no_partners

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,9.0,15.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Fri May 25 19:30:28 UTC 2007,Wed Oct 30 00:07:05 UTC 2013,"<p>Zoho offers a suite of Business, Collaborat...","{'available_sizes': [[[150, 55], 'assets/image...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,NaN,NaN,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,2007-05-25 19:24:22,Wed Oct 31 18:26:09 UTC 2012,"<p>AdventNet is now <a href=""/company/zoho-man...","{'available_sizes': [[[150, 55], 'assets/image...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,NaN,NaN,NaN,NaN,"flektor, photo, video",None,None,None,None,Thu May 31 21:11:51 UTC 2007,Sat Nov 05 08:42:23 UTC 2011,<p>Flektor is a rich-media mash-up platform th...,"{'available_sizes': [[[150, 68], 'assets/image...","[{'name': 'Flektor', 'permalink': 'flektor'}]","[{'is_past': True, 'title': 'COO, Co-Founder',...","[{'competitor': {'name': 'RockYou', 'permalink...",[],$0,[],[],"{'price_amount': 20000000, 'price_currency_cod...",[],"[{'description': None, 'address1': '8536 Natio...",[],[],[],[],[],NaN,NaN,NaN,NaN


### 7. All the companies that have a null type of value on the `category_code` field.

In [63]:
category_code_null = pd.DataFrame(companies.find({'category_code': {'$type': 'null'}}).limit(3))
category_code_null

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd6752980f6,Collective,collective,http://www.crunchbase.com/company/collective,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Thu Sep 26 13:15:02 UTC 2013,Thu Sep 26 13:15:02 UTC 2013,None,None,[],[],[],[],$0,[],[],None,[],[],[],None,[],[],[],[]
1,52cdef7c4bab8bd675298225,Snimmer,snimmer,http://www.crunchbase.com/company/snimmer,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Sat Feb 09 20:35:27 UTC 2008,Mon Mar 10 01:30:14 UTC 2008,None,None,[],[],[],[],$0,[],[],None,[],[],[],None,[],[],[],[]
2,52cdef7c4bab8bd675298226,KoolIM,koolim,http://www.crunchbase.com/company/koolim,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,None,None,None,Sat Feb 09 20:35:27 UTC 2008,Mon Nov 29 19:02:26 UTC 2010,<p>KoolIM.com is a web messenger application t...,None,[],"[{'is_past': True, 'title': 'Co-Founder', 'per...","[{'competitor': {'name': 'eBuddy', 'permalink'...",[],$0,[],[],None,[],[],[],None,[],[],[],[]


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [64]:
cond1 = {'number_of_employees': {'$gte':100}}
cond2 = {'number_of_employees': {'$lt':1000}}
employees_100_100 =  pd.DataFrame(companies.find({'$and': [cond1, cond2]}, 
                                                 {'name':1, 'number_of_employees':1, '_id':0}).limit(3))
employees_100_100

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305


### 9. Order all the companies by their IPO price in a descending order.

In [69]:
ipo_order = pd.DataFrame(companies.find().sort('ipo', -1).limit(3))
ipo_order

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7e4bab8bd67529a8b4,GREE,gree,http://www.crunchbase.com/company/gree,http://www.gree-corp.com,,,gree_corp,games_video,700.0,2004,12.0,7.0,None,None,None,None,"mobile-web, japan, tokyo, social-network, mobi...",None,inquiry@gree-corp.com,,"Internet media business,SNS, free game",Sat Dec 20 16:42:57 UTC 2008,Tue Jan 01 21:37:04 UTC 2013,<p>GREE provides Japan&#8217;s leading mobile ...,"{'available_sizes': [[[150, 61], 'assets/image...",[],"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Mobage-town', 'perma...",[],¥464M,"[{'id': 18872, 'round_code': 'unattributed', '...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': 104000000, 'price_currency_c...","[{'description': 'Headquarters', 'address1': '...","[{'id': 1021, 'description': 'IPO', 'stoned_ye...","{'valuation_amount': 108960000000, 'valuation_...",[],"[{'available_sizes': [[[133, 150], 'assets/ima...",[],[]
1,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,2004,2.0,1.0,None,None,None,,"facebook, college, students, profiles, network...",,,,Social network,Fri May 25 21:22:15 UTC 2007,Thu Nov 21 19:40:55 UTC 2013,<p>Facebook is the world&#8217;s largest socia...,"{'available_sizes': [[[150, 61], 'assets/image...","[{'name': 'Facebook Platform', 'permalink': 'f...","[{'is_past': False, 'title': 'Founder and CEO,...","[{'competitor': {'name': 'MySpace', 'permalink...","[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarters', 'address1': '...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...",[],"[{'available_sizes': [[[150, 68], 'assets/imag...",[{'external_url': 'http://latimesblogs.latimes...,[]
2,52cdef7c4bab8bd675297e7a,Amazon,amazon,http://www.crunchbase.com/company/amazon,http://amazon.com,,,amazon,ecommerce,NaN,1994,NaN,NaN,None,None,None,None,"virtualstorage, onlineshopping, virtualserver,...",,,(206) 266-1000,,Tue Jul 31 06:46:49 UTC 2007,Wed Oct 30 17:13:43 UTC 2013,"<p>Amazon.com, Inc. (AMZN), is a leading globa...","{'available_sizes': [[[150, 66], 'assets/image...","[{'name': 'Amazon EC2', 'permalink': 'amazon-e...","[{'is_past': False, 'title': 'Founder, Preside...","[{'competitor': {'name': 'eMusic', 'permalink'...","[{'title': '', 'is_past': False, 'provider': {...",$8M,"[{'id': 24535, 'round_code': 'a', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': '1200 12th ...","[{'id': 6931, 'description': 'How Many Kindle ...","{'valuation_amount': 100000000000, 'valuation_...","[{'embed_code': '<object width=""425"" height=""3...","[{'available_sizes': [[[150, 94], 'assets/imag...",[],[]


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [70]:
more_employees = pd.DataFrame(companies.find().sort('number_of_employees', -1).limit(10))
more_employees

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,1847,NaN,NaN,None,None,None,None,"automation, building-technologies, drive-techn...",,contact@siemens.com,49 89 636 34134,Electronics and Electrical Engineering,Thu Jul 31 09:29:43 UTC 2008,Thu Nov 28 20:32:55 UTC 2013,"<p>Siemens AG, an electronics and electrical e...","{'available_sizes': [[[150, 34], 'assets/image...",[],"[{'is_past': False, 'title': 'Chairman, Superv...",[],"[{'title': 'Application Development', 'is_past...",$8.9M,"[{'id': 17062, 'round_code': 'grant', 'source_...",[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,1896,NaN,NaN,None,None,None,,,,ews@us.ibm.com,914-499-1900,,Fri Mar 14 22:55:52 UTC 2008,Sat Jan 04 02:56:24 UTC 2014,"<p>IBM, acronym for International Business Mac...","{'available_sizes': [[[150, 60], 'assets/image...","[{'name': 'Lotus Notes', 'permalink': 'lotus-n...","[{'is_past': False, 'title': 'Chairman of the ...","[{'competitor': {'name': 'BEA Systems', 'perma...","[{'title': '', 'is_past': False, 'provider': {...",$0,[],"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]
2,52cdef7d4bab8bd675299d33,Toyota,toyota,http://www.crunchbase.com/company/toyota,http://www.toyota-global.com,,,Toyota,enterprise,320000,1933,NaN,NaN,None,None,None,None,None,,,81 5 6528 2121,Automotive,Mon Sep 22 22:00:26 UTC 2008,Mon Nov 11 16:24:21 UTC 2013,<p>Toyota operates 75 manufacturing companies ...,"{'available_sizes': [[[135, 37], 'assets/image...",[],"[{'is_past': False, 'title': '', 'person': {'f...",[],"[{'title': '', 'is_past': True, 'provider': {'...",$0,[],"[{'funding_round': {'round_code': 'a', 'source...",None,[],"[{'description': 'HQ', 'address1': 'Toyota Mot...",[],None,"[{'embed_code': '<iframe width=""430"" height=""3...","[{'available_sizes': [[[150, 84], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000,1998,12.0,1.0,None,None,None,None,"moneytransfer, crunchbase, ecommerce, paypal, ...",,,207-619-2873,Internet payment service,Wed Aug 01 09:16:46 UTC 2007,Fri Dec 20 04:43:53 UTC 2013,<p>PayPal is an online payments and money tran...,"{'available_sizes': [[[150, 51], 'assets/image...","[{'name': 'PayPal Here', 'permalink': 'paypal-...","[{'is_past': False, 'title': 'Vice President, ...","[{'competitor': {'name': 'KushCash', 'permalin...","[{'title': 'Public Relations', 'is_past': True...",$197M,"[{'id': 284, 'round_code': 'd', 'source_url': ...","[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'descr

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
founded_2_ses = pd.DataFrame(companies.find({'founded_month': {'$gt':6}}).limit(1000))
founded_2_ses

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [75]:
cond1 = {'founded_year': {'$lt':2000}}
cond2 = {'acquisition.price_amount': {'$gt':1000000}}
found_aq = pd.DataFrame(companies.find({'$and': [cond1, cond2]}).limit(3))
found_aq

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,1999,6,2,None,None,None,None,,None,,888.584.3150,None,Fri Jun 08 12:19:51 UTC 2007,Sat Aug 13 18:02:34 UTC 2011,<p>Postini focuses on two main issues: securit...,"{'available_sizes': [[[150, 61], 'assets/image...","[{'name': 'Postini Solutions', 'permalink': 'p...","[{'is_past': False, 'title': 'Board', 'person'...","[{'competitor': {'name': 'RescueTime', 'permal...",[],$0,[],[],"{'price_amount': 625000000, 'price_currency_co...",[],"[{'description': None, 'address1': '959 Skyway...",[],None,[],[],[],[]
1,52cdef7c4bab8bd675297deb,SideStep,sidestep,http://www.crunchbase.com/company/sidestep,http://www.sidestep.com,None,None,None,web,75.0,1999,11,1,None,None,None,None,"deals, travel, flights, hotels, tickets, vacat...",None,None,408-235-1700,None,Wed Jul 11 02:50:10 UTC 2007,Thu Jan 13 02:41:17 UTC 2011,<p>SideStep is a metasearch engine for travel ...,"{'available_sizes': [[[150, 52], 'assets/image...","[{'name': 'SideStep', 'permalink': 'sidestep'}]","[{'is_past': False, 'title': 'Board', 'person'...","[{'competitor': {'name': 'Mobissimo', 'permali...",[],$32M,"[{'id': 105, 'round_code': 'c', 'source_url': ...",[],"{'price_amount': 180000000, 'price_currency_co...",[],"[{'description': None, 'address1': '2980 Bower...",[],None,[],[],[],[]
2,52cdef7c4bab8bd675297e2c,Recipezaar,recipezaar,http://www.crunchbase.com/company/recipezaar,http://www.recipezaar.com,http://www.recipezaar.com/sitenews/,None,None,web,5.0,1999,9,1,None,None,None,None,"recipes, recipezaar, cooking, cookpoint",None,None,None,None,Fri Jul 20 03:47:19 UTC 2007,Tue Feb 14 08:27:43 UTC 2012,"<p>Launched in December 1999, Recipezaar (orig...","{'available_sizes': [[[147, 100], 'assets/imag...","[{'name': 'Recipezaar', 'permalink': 'recipeza...","[{'is_past': None, 'title': 'Co-Founder', 'per...","[{'competitor': {'name': 'Cookthink', 'permali...",[],$0,[],[],"{'price_amount': 25000000, 'price_currency_cod...",[],"[{'description': None, 'address1': 'P.O. Box 1...",[],None,[],[],[],[]


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code